In [4]:
import numpy as np
import matplotlib.pyplot as plt
from helpers import *
from cleaning_data import *
from stats import *
from functions import *
from clean_data_testing import *
from utils import split_data, stratified_train_test_split, downsample_class_0, upsample_class_1_to_percentage
import datetime
import seaborn as sns
%load_ext autoreload
%autoreload 2

PermissionError: [Errno 1] Operation not permitted

In [3]:
from functions import *

PermissionError: [Errno 1] Operation not permitted

In [46]:
from clean_data_testing import *

In [47]:
DATA_PATH = '/Users/williamjallot/Desktop/ML/dataset'
x_train, x_test, y_train, train_ids, test_ids, labels =  load_csv_data(DATA_PATH, sub_sample=False)
labels.pop(0) 

'Id'

In [48]:
x_tr, x_te, y_tr, y_te = split_data(x_train, y_train, 0.8, seed= 2)

In [49]:
y_tr[y_tr == -1] = 0

In [50]:
def calculate_downsample_size(N_pos, N_neg, desired_percentage):
    # Calculate the required downsample size for the negative class
    downsample_size_neg = int((N_pos * (1 - desired_percentage)) / desired_percentage)
    
    # Ensure we do not downsample more than available negative samples
    downsample_size_neg = min(downsample_size_neg, N_neg)
    
    return downsample_size_neg

In [51]:
N_pos = len(np.where(y_tr == 1)[0])
N_neg = len(np.where(y_tr == 0)[0])

In [52]:
down_sampling_size = calculate_downsample_size(N_pos, N_neg, 0.25)

In [53]:
x_tr_up, y_tr_up = upsample_class_1_to_percentage(x_tr, y_tr, 0.3)

In [54]:
x_tr_ds, y_tr_ds = downsample_class_0(x_tr, y_tr, down_sampling_size)

In [55]:
x_train_cleaned, features, median_and_most_probable_class, W, mean = clean_data_x(x_tr_up, labels)

    
np.savetxt("/Users/williamjallot/Desktop/ML/dataset/x_trained.cleaned.csv", x_train_cleaned, delimiter=",")

Number of components to retain 90.0% variance: 63


In [56]:
x_te_cleaned = clean_data_testing(x_te, labels, features, median_and_most_probable_class, mean, W)

In [57]:
x_test_cleaned = clean_data_testing(x_test, labels, features, median_and_most_probable_class, mean, W)

In [58]:
x_te_cleaned.shape

(65627, 63)

In [59]:
x_train_cleaned.shape

(342081, 63)

In [60]:
max_iters = 100000
gamma = 0.001
batch_size = 64
lambda_ =  0.005
num_samples = x_train_cleaned.shape[0]



# Parameters for the Gaussian distribution
mean = 0    # Mean of the distribution
std_dev = 1 # Standard deviation of the distribution

tx = np.c_[np.ones(num_samples), x_train_cleaned]
w_initial = np.random.normal(loc=mean, scale=std_dev, size=tx.shape[1])


sgd_losses, sgd_ws = stochastic_gradient_descent(
y_tr_up, tx, w_initial, batch_size, max_iters, gamma, lambda_
)


In [61]:
num_samples = x_te.shape[0]
tx_te = np.c_[np.ones(num_samples), x_te_cleaned]
y_predict = predict(tx_te, sgd_ws[-1])

In [62]:
accuracy(y_te, y_predict)

0.8003870358236701

In [63]:
f1_score(y_te, y_predict)

(0.38190053788808154, 0.6831532748143146)

In [66]:
num_samples = x_test_cleaned.shape[0]
tx_test = np.c_[np.ones(num_samples), x_test_cleaned]
y_test_to_save = predict(tx_test, sgd_ws[-1])

In [77]:
np.savetxt("/Users/williamjallot/Desktop/ML/dataset/sample_submission.csv", y_test_to_save, delimiter=",")

In [79]:
len(test_ids)

109379

In [80]:
len(y_test_to_save)

109379

In [82]:
# Stack the ids and predictions together column-wise
submit = np.column_stack((test_ids, y_test_to_save))

# Save to a CSV file using np.savetxt
np.savetxt("/Users/williamjallot/Desktop/ML/dataset/sample_submission.csv", submit, delimiter=",", fmt='%d,%d', header='Id,Prediction', comments='')

In [1]:
np.sum(4)

NameError: name 'np' is not defined